<a href="https://colab.research.google.com/github/danielsuassuna04/Resnet50-with-keras/blob/main/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

from tensorflow import keras

In [ ]:
def data_augmentation():
    data_augmentation = keras.models.Sequential()
    data_augmentation.add(keras.layers.RandomFlip('horizontal'))
    data_augmentation.add(keras.layers.RandomRotation(0.2))
    data_augmentation.add(keras.layers.RandomZoom(0.2))
    data_augmentation.add(keras.layers.RandomContrast(0.2))



    return data_augmentation

In [ ]:
def bloco_identidade(X, f, filters, initializer=keras.initializers.random_uniform):
    F1, F2, F3 = filters

    X_shortcut = X

    X = keras.layers.Conv2D(filters=F1, kernel_size=1, strides=(1,1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = keras.layers.BatchNormalization(axis=3)(X)
    X = keras.layers.Activation('relu')(X)

    X = keras.layers.Conv2D(filters=F2, kernel_size=(f, f), strides=(1,1), padding='same', kernel_initializer=initializer(seed=0))(X)
    X = keras.layers.BatchNormalization(axis=3)(X)
    X = keras.layers.Activation('relu')(X)

    X = keras.layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=(1,1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = keras.layers.BatchNormalization(axis=3)(X)

    X = keras.layers.Add()([X, X_shortcut])
    X = keras.layers.Activation('relu')(X)

    return X


In [ ]:
def bloco_convolucional(X, f, filters, s=2, initializer=keras.initializers.glorot_uniform):
    F1, F2, F3 = filters
    X_shortcut = X

    X = keras.layers.Conv2D(filters=F1, kernel_size=1, strides=(s, s), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = keras.layers.BatchNormalization(axis=3)(X)
    X = keras.layers.Activation('relu')(X)

    X = keras.layers.Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', kernel_initializer=initializer(seed=0))(X)
    X = keras.layers.BatchNormalization(axis=3)(X)
    X = keras.layers.Activation('relu')(X)

    X = keras.layers.Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = keras.layers.BatchNormalization(axis=3)(X)

    X_shortcut = keras.layers.Conv2D(F3, (1, 1), strides=(s, s), kernel_initializer=initializer(seed=0))(X_shortcut)
    X_shortcut = keras.layers.BatchNormalization(axis=3)(X_shortcut)

    X = keras.layers.Add()([X, X_shortcut])
    X = keras.layers.Activation('relu')(X)

    return X


In [ ]:
def ResNet50(input_shape=(64, 64, 3), classes=6, training=False):
    X_input = keras.layers.Input(input_shape)
    X_input = data_augmentation()(X_input)
    X = keras.layers.ZeroPadding2D((3, 3))(X_input)

    X = keras.layers.Conv2D(64, (7, 7), strides=(2, 2), kernel_initializer=keras.initializers.glorot_uniform(seed=0))(X)
    X = keras.layers.BatchNormalization(axis=3)(X)
    X = keras.layers.Activation('relu')(X)
    X = keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = bloco_convolucional(X, f=3, filters=[64, 64, 256], s=1)
    X = bloco_identidade(X, 3, [64, 64, 256])
    X = bloco_identidade(X, 3, [64, 64, 256])

    X = bloco_convolucional(X, f=3, filters=[128, 128, 512], s=2)
    X = bloco_identidade(X, f=3, filters=[128, 128, 512])
    X = bloco_identidade(X, f=3, filters=[128, 128, 512])
    X = bloco_identidade(X, f=3, filters=[128, 128, 512])

    X = bloco_convolucional(X, f=3, filters=[256, 256, 1024], s=2)
    X = bloco_identidade(X, f=3, filters=[256, 256, 1024])
    X = bloco_identidade(X, f=3, filters=[256, 256, 1024])
    X = bloco_identidade(X, f=3, filters=[256, 256, 1024])
    X = bloco_identidade(X, f=3, filters=[256, 256, 1024])
    X = bloco_identidade(X, f=3, filters=[256, 256, 1024])

    X = bloco_convolucional(X, f=3, filters=[512, 512, 2048], s=2)
    X = bloco_identidade(X, f=3, filters=[512, 512, 2048])
    X = bloco_identidade(X, f=3, filters=[512, 512, 2048])

    X = keras.layers.AveragePooling2D((2, 2))(X)

    X = keras.layers.Flatten()(X)
    X = keras.layers.Dropout(0.5)(X)
    X = keras.layers.Dense(classes, activation='softmax', kernel_initializer=keras.initializers.glorot_uniform(seed=0))(X)

    model = keras.Model(inputs=X_input, outputs=X)

    return model


In [ ]:
from tensorflow.keras.applications.resnet import preprocess_input


(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

batch_size = 64

def preprocess(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (64, 64))
    return image, label
X_train = train_images.astype('float32') / 255.0
X_test = test_images.astype('float32') / 255.0

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

train_dataset = (train_dataset
                 .map(lambda x, y: preprocess(x, y))
                 .shuffle(buffer_size=10000)
                 .batch(batch_size)
                 .prefetch(1))
test_dataset = (test_dataset
                .map(lambda x, y: preprocess(x, y))
                .batch(batch_size)
                .prefetch(1))

In [ ]:
model = ResNet50(input_shape=(64, 64, 3), classes=10)

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01,weight_decay=0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath='best_model.weights.h5',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    save_weights_only=True,
    verbose=1
)

In [ ]:
model.fit(train_dataset, epochs=30, validation_data=test_dataset,callbacks=[checkpoint])

Epoch 1/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 0.6863 - loss: 0.9087
Epoch 1: val_loss improved from inf to 1.36041, saving model to best_model.weights.h5
782/782 ━━━━━━━━━━━━━━━━━━━━ 161s 122ms/step - accuracy: 0.6864 - loss: 0.9087 - val_accuracy: 0.5605 - val_loss: 1.3604
Epoch 2/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.7157 - loss: 0.8226
Epoch 2: val_loss improved from 1.36041 to 1.04611, saving model to best_model.weights.h5
782/782 ━━━━━━━━━━━━━━━━━━━━ 84s 85ms/step - accuracy: 0.7157 - loss: 0.8226 - val_accuracy: 0.6362 - val_loss: 1.0461
Epoch 3/30
781/782 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.7316 - loss: 0.7755
Epoch 3: val_loss improved from 1.04611 to 0.91440, saving model to best_model.weights.h5
782/782 ━━━━━━━━━━━━━━━━━━━━ 67s 85ms/step - accuracy: 0.7316 - loss: 0.7754 - val_accuracy: 0.6943 - val_loss: 0.9144
Epoch 4/30
781/782 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.7584 - loss: 0.7141
Epoch 4: val_loss did not 

In [ ]:
!git config user.email "danielsuassuna05@gmail.com"
!git config user.name "danielsuasuna

/bin/bash: -c: line 1: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 2: syntax error: unexpected end of file


In [ ]:
!git add ResNet.ipynb
!git commit -m "ResNet"
!git push

[main (root-commit) ebefacd] ResNet
 1 file changed, 434 insertions(+)
 create mode 100644 ResNet.ipynb
fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
!dir

ResNet.ipynb
